In [2]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [3]:
def collecting_info(brutto_lohn_jahr: str="30000", abrechnungs_jahr: str="2024", steuerklasse: str="1"):
    url = 'https://www.brutto-netto-rechner.info/index.php'
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
        'accept-language': 'en-US,en;q=0.8',
        'cache-control': 'max-age=0',
        'content-type': 'application/x-www-form-urlencoded',
        'origin': 'https://www.brutto-netto-rechner.info',
        'referer': 'https://www.brutto-netto-rechner.info/',
        'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Brave";v="122"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'sec-gpc': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
    }
    data = {
        'f_start': '1',
        'ok': '1',
        'f_bruttolohn': f'{brutto_lohn_jahr}',
        'f_abrechnungszeitraum': 'jahr',
        'f_geld_werter_vorteil': '0',
        'f_abrechnungsjahr': f'{abrechnungs_jahr}',
        'f_steuerfreibetrag': '',
        'f_steuerklasse': f'{steuerklasse}',
        'f_kirche': 'nein',
        'f_bundesland': 'baden-wuerttemberg',
        'f_alter': '35',
        'f_kinder': 'nein',
        'f_kinderfreibetrag': '0',
        'f_kinder_anz': '0',
        'f_krankenversicherung': 'pflichtversichert',
        'f_private_kv': '',
        'f_arbeitgeberzuschuss_pkv': 'ja',
        'f_KVZ': '1.7',
        'f_rentenversicherung': 'pflichtversichert',
        'f_arbeitslosenversicherung': 'pflichtversichert'
    }

    response = requests.post(url, headers=headers, data=data)
    return response

In [4]:
response = collecting_info(brutto_lohn_jahr="30000", abrechnungs_jahr="2024", steuerklasse="1")

In [5]:
soup = bs(response.content, 'lxml')

table = soup.find('table', {'class': 'rechner'})

In [18]:
table_data = []
for row in table.find_all("tr"):
    data = row.find_all("td")
    small_list = []
    for info in data:
        if info.text.strip():
            small_list.append(info.text.strip())
    table_data.append(small_list)
table_data

[['Ergebnis', 'Monat', 'Jahr'],
 [],
 ['Brutto:', '2.500,00\xa0€', '30.000,00\xa0€'],
 ['Geldwerter\xa0Vorteil:', '0,00\xa0€', '0,00\xa0€'],
 [],
 ['Steuern'],
 ['Solidaritäts-\xa0zuschlag:', '0,00\xa0€', '0,00\xa0€'],
 ['Kirchensteuer:', '0,00\xa0€', '0,00\xa0€'],
 ['Lohnsteuer:', '208,41\xa0€', '2.500,92\xa0€'],
 [],
 ['Steuern:', '208,41\xa0€', '2.500,92\xa0€'],
 [],
 ['Sozialabgaben'],
 ['Renten-\xa0versicherung:', '232,50\xa0€', '2.790,00\xa0€'],
 ['Bessere Absicherung und Steuerersparnis mit der Riester-Rente! Bis zu 2.100,- €\n2.100,- € sind steuerlich abzugsfähig: Infos anfordern!Infos anfordern!'],
 ['Arbeitslosen-\xa0versicherung:', '32,50\xa0€', '390,00\xa0€'],
 ['Kranken-\xa0versicherung:', '203,75\xa0€', '2.445,00\xa0€'],
 ['Pflege-\xa0versicherung:', '57,50 €', '690,00 €'],
 [],
 ['Sozialabgaben:', '526,25\xa0€', '6.315,00\xa0€'],
 [],
 ['Netto:', '1.765,34\xa0€', '21.184,08\xa0€'],
 [],
 [],
 [],
 []]

In [19]:
clean_table_data = []

for row in table_data:
    clean_row = []
    for cell in row:
        # Replace non-breaking spaces with regular spaces and strip leading/trailing whitespace
        clean_cell = cell.replace('\xa0', ' ').strip()
        clean_row.append(clean_cell)
    # Filter out empty strings if the entire row isn't empty
    if any(clean_row):
        clean_row = [cell for cell in clean_row if cell != '']
        clean_table_data.append(clean_row)
clean_table_data

[['Ergebnis', 'Monat', 'Jahr'],
 ['Brutto:', '2.500,00 €', '30.000,00 €'],
 ['Geldwerter Vorteil:', '0,00 €', '0,00 €'],
 ['Steuern'],
 ['Solidaritäts- zuschlag:', '0,00 €', '0,00 €'],
 ['Kirchensteuer:', '0,00 €', '0,00 €'],
 ['Lohnsteuer:', '208,41 €', '2.500,92 €'],
 ['Steuern:', '208,41 €', '2.500,92 €'],
 ['Sozialabgaben'],
 ['Renten- versicherung:', '232,50 €', '2.790,00 €'],
 ['Bessere Absicherung und Steuerersparnis mit der Riester-Rente! Bis zu 2.100,- €\n2.100,- € sind steuerlich abzugsfähig: Infos anfordern!Infos anfordern!'],
 ['Arbeitslosen- versicherung:', '32,50 €', '390,00 €'],
 ['Kranken- versicherung:', '203,75 €', '2.445,00 €'],
 ['Pflege- versicherung:', '57,50 €', '690,00 €'],
 ['Sozialabgaben:', '526,25 €', '6.315,00 €'],
 ['Netto:', '1.765,34 €', '21.184,08 €']]

In [20]:
for i, row in enumerate(clean_table_data):
    if len(row) != 3:
        del clean_table_data[i]
clean_table_data

[['Ergebnis', 'Monat', 'Jahr'],
 ['Brutto:', '2.500,00 €', '30.000,00 €'],
 ['Geldwerter Vorteil:', '0,00 €', '0,00 €'],
 ['Solidaritäts- zuschlag:', '0,00 €', '0,00 €'],
 ['Kirchensteuer:', '0,00 €', '0,00 €'],
 ['Lohnsteuer:', '208,41 €', '2.500,92 €'],
 ['Steuern:', '208,41 €', '2.500,92 €'],
 ['Renten- versicherung:', '232,50 €', '2.790,00 €'],
 ['Arbeitslosen- versicherung:', '32,50 €', '390,00 €'],
 ['Kranken- versicherung:', '203,75 €', '2.445,00 €'],
 ['Pflege- versicherung:', '57,50 €', '690,00 €'],
 ['Sozialabgaben:', '526,25 €', '6.315,00 €'],
 ['Netto:', '1.765,34 €', '21.184,08 €']]

In [21]:
df = pd.DataFrame(clean_table_data)
df

,0,1,2
0,Ergebnis,Monat,Jahr
1,Brutto:,"2.500,00 €","30.000,00 €"
2,Geldwerter Vorteil:,"0,00 €","0,00 €"
3,Solidaritäts- zuschlag:,"0,00 €","0,00 €"
4,Kirchensteuer:,"0,00 €","0,00 €"
5,Lohnsteuer:,"208,41 €","2.500,92 €"
6,Steuern:,"208,41 €","2.500,92 €"
7,Renten- versicherung:,"232,50 €","2.790,00 €"
8,Arbeitslosen- versicherung:,"32,50 €","390,00 €"
9,Kranken- versicherung:,"203,75 €","2.445,00 €"


In [27]:
df  = df.transpose()

In [28]:
df.columns = df.iloc[0]
df = df[1:]
df

,Ergebnis,Brutto:,Geldwerter Vorteil:,Solidaritäts- zuschlag:,Kirchensteuer:,Lohnsteuer:,Steuern:,Renten- versicherung:,Arbeitslosen- versicherung:,Kranken- versicherung:,Pflege- versicherung:,Sozialabgaben:,Netto:
1,Monat,"2.500,00 €","0,00 €","0,00 €","0,00 €","208,41 €","208,41 €","232,50 €","32,50 €","203,75 €","57,50 €","526,25 €","1.765,34 €"
2,Jahr,"30.000,00 €","0,00 €","0,00 €","0,00 €","2.500,92 €","2.500,92 €","2.790,00 €","390,00 €","2.445,00 €","690,00 €","6.315,00 €","21.184,08 €"


In [30]:
new_data = {
    # Brutto
    "Brutto Monat": df.loc[df["Ergebnis"] == "Monat", "Brutto:"].values,
    "Brutto Jahr": df.loc[df["Ergebnis"] == "Jahr", "Brutto:"].values,
    
    # Geldwerter Vorteil
    "Geldwerter Vorteil Monat": df.loc[df["Ergebnis"] == "Monat", "Geldwerter Vorteil:"].values,
    "Geldwerter Vorteil Jahr": df.loc[df["Ergebnis"] == "Jahr", "Geldwerter Vorteil:"].values,
    
    # Solidaritätszuschlag
    "Solidaritätszuschlag Monat": df.loc[df["Ergebnis"] == "Monat", "Solidaritäts- zuschlag:"].values,
    "Solidaritätszuschlag Jahr": df.loc[df["Ergebnis"] == "Jahr", "Solidaritäts- zuschlag:"].values,
    
    # Kirchensteuer
    "Kirchensteuer Monat": df.loc[df["Ergebnis"] == "Monat", "Kirchensteuer:"].values,
    "Kirchensteuer Jahr": df.loc[df["Ergebnis"] == "Jahr", "Kirchensteuer:"].values,
    
    # Lohnsteuer
    "Lohnsteuer Monat": df.loc[df["Ergebnis"] == "Monat", "Lohnsteuer:"].values,
    "Lohnsteuer Jahr": df.loc[df["Ergebnis"] == "Jahr", "Lohnsteuer:"].values,
    
    # Steuern
    "Steuern Monat": df.loc[df["Ergebnis"] == "Monat", "Steuern:"].values,
    "Steuern Jahr": df.loc[df["Ergebnis"] == "Jahr", "Steuern:"].values,
    
    # Rentenversicherung
    "Rentenversicherung Monat": df.loc[df["Ergebnis"] == "Monat", "Renten- versicherung:"].values,
    "Rentenversicherung Jahr": df.loc[df["Ergebnis"] == "Jahr", "Renten- versicherung:"].values,
    
    # Arbeitslosenversicherung
    "Arbeitslosenversicherung Monat": df.loc[df["Ergebnis"] == "Monat", "Arbeitslosen- versicherung:"].values,
    "Arbeitslosenversicherung Jahr": df.loc[df["Ergebnis"] == "Jahr", "Arbeitslosen- versicherung:"].values,
    
    # Krankenversicherung
    "Krankenversicherung Monat": df.loc[df["Ergebnis"] == "Monat", "Kranken- versicherung:"].values,
    "Krankenversicherung Jahr": df.loc[df["Ergebnis"] == "Jahr", "Kranken- versicherung:"].values,
    
    # Pflegeversicherung
    "Pflegeversicherung Monat": df.loc[df["Ergebnis"] == "Monat", "Pflege- versicherung:"].values,
    "Pflegeversicherung Jahr": df.loc[df["Ergebnis"] == "Jahr", "Pflege- versicherung:"].values,
    
    # Sozialabgaben
    "Sozialabgaben Monat": df.loc[df["Ergebnis"] == "Monat", "Sozialabgaben:"].values,
    "Sozialabgaben Jahr": df.loc[df["Ergebnis"] == "Jahr", "Sozialabgaben:"].values,
    
    # Netto
    "Netto Monat": df.loc[df["Ergebnis"] == "Monat", "Netto:"].values,
    "Netto Jahr": df.loc[df["Ergebnis"] == "Jahr", "Netto:"].values
}


In [32]:
new_data

{'Brutto Monat': array(['2.500,00 €'], dtype=object),
 'Brutto Jahr': array(['30.000,00 €'], dtype=object),
 'Geldwerter Vorteil Monat': array(['0,00 €'], dtype=object),
 'Geldwerter Vorteil Jahr': array(['0,00 €'], dtype=object),
 'Solidaritätszuschlag Monat': array(['0,00 €'], dtype=object),
 'Solidaritätszuschlag Jahr': array(['0,00 €'], dtype=object),
 'Kirchensteuer Monat': array(['0,00 €'], dtype=object),
 'Kirchensteuer Jahr': array(['0,00 €'], dtype=object),
 'Lohnsteuer Monat': array(['208,41 €'], dtype=object),
 'Lohnsteuer Jahr': array(['2.500,92 €'], dtype=object),
 'Steuern Monat': array(['208,41 €'], dtype=object),
 'Steuern Jahr': array(['2.500,92 €'], dtype=object),
 'Rentenversicherung Monat': array(['232,50 €'], dtype=object),
 'Rentenversicherung Jahr': array(['2.790,00 €'], dtype=object),
 'Arbeitslosenversicherung Monat': array(['32,50 €'], dtype=object),
 'Arbeitslosenversicherung Jahr': array(['390,00 €'], dtype=object),
 'Krankenversicherung Monat': array(['203,

In [35]:
new_df = pd.DataFrame(new_data, index=["Values"])

In [36]:
new_df

,Brutto Monat,Brutto Jahr,Geldwerter Vorteil Monat,Geldwerter Vorteil Jahr,Solidaritätszuschlag Monat,Solidaritätszuschlag Jahr,Kirchensteuer Monat,Kirchensteuer Jahr,Lohnsteuer Monat,Lohnsteuer Jahr,...,Arbeitslosenversicherung Monat,Arbeitslosenversicherung Jahr,Krankenversicherung Monat,Krankenversicherung Jahr,Pflegeversicherung Monat,Pflegeversicherung Jahr,Sozialabgaben Monat,Sozialabgaben Jahr,Netto Monat,Netto Jahr
Values,"2.500,00 €","30.000,00 €","0,00 €","0,00 €","0,00 €","0,00 €","0,00 €","0,00 €","208,41 €","2.500,92 €",...,"32,50 €","390,00 €","203,75 €","2.445,00 €","57,50 €","690,00 €","526,25 €","6.315,00 €","1.765,34 €","21.184,08 €"


In [43]:
type(new_df.iloc[0][0])

C:\Users\oeztano\AppData\Local\Temp\ipykernel_15352\3308931940.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  type(new_df.iloc[0][0])


str

In [44]:
import pandas as pd

# Assuming `new_df` is your DataFrame and it contains currency values as strings
for column in new_df.columns:
    # Remove currency symbol (€) and thousands separator (.), and replace comma with dot for decimal
    new_df[column] = new_df[column].str.replace('€', '').str.replace('.', '').str.replace(',', '.').astype(float)


In [45]:
new_df

,Brutto Monat,Brutto Jahr,Geldwerter Vorteil Monat,Geldwerter Vorteil Jahr,Solidaritätszuschlag Monat,Solidaritätszuschlag Jahr,Kirchensteuer Monat,Kirchensteuer Jahr,Lohnsteuer Monat,Lohnsteuer Jahr,...,Arbeitslosenversicherung Monat,Arbeitslosenversicherung Jahr,Krankenversicherung Monat,Krankenversicherung Jahr,Pflegeversicherung Monat,Pflegeversicherung Jahr,Sozialabgaben Monat,Sozialabgaben Jahr,Netto Monat,Netto Jahr
Values,2500.0,30000.0,0.0,0.0,0.0,0.0,0.0,0.0,208.41,2500.92,...,32.5,390.0,203.75,2445.0,57.5,690.0,526.25,6315.0,1765.34,21184.08


In [46]:
type(new_df.iloc[0][0])

C:\Users\oeztano\AppData\Local\Temp\ipykernel_15352\3308931940.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  type(new_df.iloc[0][0])


numpy.float64

In [10]:
import itertools
# Define your lists of search terms
brutto_lohn_jahr = [x for x in range(30000, 120000, 1000)]
abrechnungsjahr = ['2023', '2024']
steuerklasse = ['1', '3']

# Use itertools.product to generate all combinations of search terms
combinations = list(itertools.product(brutto_lohn_jahr, abrechnungsjahr, steuerklasse))

In [12]:
combinations[0][2]

'1'

In [51]:
for i, combo in enumerate(combinations[:3]):
    print(i, combo)

0 (30000, '2023', '1')
1 (30000, '2023', '3')
2 (30000, '2024', '1')


In [8]:
df = pd.read_csv("results.csv")
df

,Unnamed: 0,Brutto Monat,Brutto Jahr,Geldwerter Vorteil Monat,Geldwerter Vorteil Jahr,Solidaritätszuschlag Monat,Solidaritätszuschlag Jahr,Kirchensteuer Monat,Kirchensteuer Jahr,Lohnsteuer Monat,...,Arbeitslosenversicherung Monat,Arbeitslosenversicherung Jahr,Krankenversicherung Monat,Krankenversicherung Jahr,Pflegeversicherung Monat,Pflegeversicherung Jahr,Sozialabgaben Monat,Sozialabgaben Jahr,Netto Monat,Netto Jahr
0,Values,2500.00,30000.0,0.0,0.0,0.00,0.00,0.0,0.0,228.83,...,32.50,390.0,203.75,2445.00,46.88,562.50,515.63,6187.50,1755.55,21066.54
1,Values,2500.00,30000.0,0.0,0.0,0.00,0.00,0.0,0.0,13.66,...,32.50,390.0,203.75,2445.00,46.88,562.50,515.63,6187.50,1970.72,23648.58
2,Values,2500.00,30000.0,0.0,0.0,0.00,0.00,0.0,0.0,208.41,...,32.50,390.0,203.75,2445.00,57.50,690.00,526.25,6315.00,1765.34,21184.08
3,Values,2500.00,30000.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,...,32.50,390.0,203.75,2445.00,57.50,690.00,526.25,6315.00,1973.75,23685.00
4,Values,2583.33,31000.0,0.0,0.0,0.00,0.00,0.0,0.0,246.91,...,33.58,403.0,210.54,2526.50,48.44,581.25,532.81,6393.75,1803.61,21643.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,Values,9833.33,118000.0,0.0,0.0,0.00,0.00,0.0,0.0,1878.00,...,98.15,1177.8,421.76,5061.15,119.03,1428.30,1341.09,16093.05,6614.25,79370.95
356,Values,9916.67,119000.0,0.0,0.0,154.03,1848.36,0.0,0.0,2800.66,...,94.90,1138.8,406.48,4877.78,93.52,1122.19,1273.80,15285.56,5688.18,68258.16
357,Values,9916.67,119000.0,0.0,0.0,0.00,0.00,0.0,0.0,2007.83,...,94.90,1138.8,406.48,4877.78,93.52,1122.19,1273.80,15285.56,6635.04,79620.48
358,Values,9916.67,119000.0,0.0,0.0,144.07,1728.84,0.0,0.0,2721.58,...,98.15,1177.8,421.76,5061.15,119.03,1428.30,1341.09,16093.05,5709.93,68519.15


In [24]:
import plotly.express as px
import pandas as pd

df = pd.read_csv("results.csv")
filtered_df = df[df['Steuerklasse'].isin(['1', '3'])]
fig = px.scatter(df, x=df["Brutto Jahr"], y=df["Netto Jahr"], color=df["Steuerklasse"], title='Value Over Time')
fig.update_yaxes(scaleanchor='x', scaleratio=1)
# max_range = max(filtered_df["Brutto Jahr"].max(), filtered_df["Netto Jahr"].max())
fig.update_xaxes(range=[20000, 120000])
fig.update_yaxes(range=[20000, 120000])
fig.show()